# AgentX Quickstart

Welcome to the XEntropy quickstart guide, your first step in exploring the world of application development with agent-based large language models (or in short, agentic llm).

XEntropy is an innovative development platform designed to elevate the ease at which developers can construct intelligent applications. Throughout this tutorial, we will navigate the following key areas:

1. **Account creation:** We guide you step-by-step in establishing your XEntropy account:the gateway to a host of agentic llm tools.

2. **Loading tools from XEntropy:** Learn how to seamlessly and efficiently import tools from the platform into your environment to kickstart your foray into developing llm agents and intelligent applications.

3. **Managing your account:** Your XEntropy account is useful for many things. Learn how to:
    - **Purchase credit**: Convenience and accessibility are core to XEntropy. Understand how to replenish your account balance in a straightforward, user-friendly manner.

   - **Get paid**: Having created applications or tools that other developers find useful? Here's how to benefit from your creations.

   - **Modify and delete tools**: As your skill level advances, so might your tools and applications. Learn how to adjust and remove tools from your portfolio.

   - **Utilisation of logs**: Track your progress and rectify mistakes by understanding how to make use of logs effectively.

   - **Finetune your own agentic llm**: Capture the essence of your application development ideation by discovering how to fine-tune your personal agentic llm to reach peak performance.

## Installation

In [1]:
# Install AgentX
from distutils.dir_util import copy_tree, remove_tree
import os
import sys

site_packages_path = [x for x in sys.path if x.endswith('site-packages')][0]
# remove any existing installation
if os.path.exists(f'{site_packages_path}/agentx'):
    remove_tree(f'{site_packages_path}/agentx')
copy_tree(f'../agentx', f'{site_packages_path}/agentx')

['/home/jay/.local/lib/python3.12/site-packages/agentx/__init__.py',
 '/home/jay/.local/lib/python3.12/site-packages/agentx/__pycache__/__init__.cpython-310.pyc',
 '/home/jay/.local/lib/python3.12/site-packages/agentx/__pycache__/agent.cpython-310.pyc',
 '/home/jay/.local/lib/python3.12/site-packages/agentx/__pycache__/bedrock_client.cpython-310.pyc',
 '/home/jay/.local/lib/python3.12/site-packages/agentx/__pycache__/client.cpython-310.pyc',
 '/home/jay/.local/lib/python3.12/site-packages/agentx/__pycache__/groupchat.cpython-310.pyc',
 '/home/jay/.local/lib/python3.12/site-packages/agentx/__pycache__/oai_client.cpython-310.pyc',
 '/home/jay/.local/lib/python3.12/site-packages/agentx/__pycache__/optimisers.cpython-310.pyc',
 '/home/jay/.local/lib/python3.12/site-packages/agentx/__pycache__/saved_agents.cpython-310.pyc',
 '/home/jay/.local/lib/python3.12/site-packages/agentx/__pycache__/schema.cpython-310.pyc',
 '/home/jay/.local/lib/python3.12/site-packages/agentx/__pycache__/tool.cpyth

In [2]:
# Install dependencies to complete this tutorial
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Create your account

In [3]:
from agentx.client import Client

In [4]:
# load environment variables
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
from agentx.tool import Tool
import os

tool_search = Tool.load(
    'xentropy--tool_search', 
    api_key=os.environ.get('XENTROPY_API_KEY')
)

# display the function call schema
tool_search.input_json_schema

from rich import print as rich_print
import json
# run the tool
rich_print(
    json.loads(
        tool_search.run(query='convert an address to geolocation')
    )
)

# Get a tool that converts address to latitude longitude coordinate
geocoding = Tool.load('xentropy--geocoding', api_key=os.environ.get('XENTROPY_API_KEY'))
# Get a tool that computes the earth surface distance between two coordinates
geodesic = Tool.load('xentropy--geodesic', api_key=os.environ.get('XENTROPY_API_KEY'))

[
    {
        'tool_id': 'xentropy--geocoding',
        'description': 'Retrieve the latitude and longitude given an address using the highly accurate Google Map 
API.',
        'score': 0.85867697
    },
    {
        'tool_id': 'xentropy--geodesic',
        'description': 'Calculate the earth surface distance between two latitude and longitude coordinate',
        'score': 0.792058647
    },
    {
        'tool_id': 'xentropy--restaurant_search',
        'description': 'Search restaurants based on food preference, location, and other user requirements. 
Examples: {"include_type": "korean_restaurant", "query": "Korean barbeque in Mountain View with spacious seating 
and good service"}',
        'score': 0.719365895
    },
    {
        'tool_id': 'xentropy--tool_search',
        'description': 'Search for relevant tools to accomplish a task.',
        'score': 0.686780155
    },
    {
        'tool_id': 'xentropy--etf_search',
        'description': 'Search for ETFs that satisfy a given description.',
        'score': 0.674237192
    },
    {
        'tool_id': 'xentropy--question_answering_by_product_brochure',
        'description': 'Base on an insurance product brochure answer the query page by page.',
        'score': 0.668316543
    }
]

## Creating Agents with the loaded tools

In [6]:
from agentx.agent import Agent
from agentx.schema import GenerationConfig, Message, Content
from typing import List

tools = [geocoding, geodesic]

generation_config = GenerationConfig(
    api_type='vertexai',
    path_to_google_service_account_json='vertex_ai_creds.json',
    google_application_credential_scope=['https://www.googleapis.com/auth/cloud-platform'],
    region='asia-northeast3',
)

""" generation_config = GenerationConfig(
    api_type='azure',
    api_key=os.environ.get('AZURE_OPENAI_KEY'),
    api_version="2024-02-15-preview",
    base_url="https://genai-instance-test-jeremy.openai.azure.com/", 
) """

# replace with your own azure deployment to choose different models
generation_config.model = 'gemini-pro'
#generation_config.azure_deployment = 'gpt-35-jeremy'

# define a termination function
# once the agent execute the geodesic tool, the conversation ends
def terminate(history:List[Message]):
    tool_response_name = [
        message.content.tool_response.name for message in history if message.content.tool_response
    ]
    if 'xentropy--geodesic' in tool_response_name:
        return True
    return False
    
# define the agent
agent = Agent(
    name='agent',
    generation_config=generation_config,
    system_prompt=None,#'Use the functions you have been provided to solve the problem. Reply TERMINATE to end the conversation when the problem is solved.',
    tools = tools,
    termination_function=terminate,
)

In [7]:
# the agent is ready to generate messages
messages = [
    Message(
        role='user',
        content=Content(
            text='What is the distance between Gare Port La Goulette - Sud in Tunisia and Porto di Napoli in Italy?',
        ),
    )
]

In [8]:
# use the agent to use geocoding and geodesic to find earth surface distance between two locations

max_iterations = 10

for i in range(max_iterations):
    print("here")
    response = agent.generate_response(messages)
    # termination condition is met
    if response == None:
        break
    else:
        [rich_print(r.model_dump(exclude_unset=True)) for r in response]
    messages += response

here
xentropy--geocoding transformed to vertex AI schema successfully
xentropy--geodesic transformed to vertex AI schema successfully


ValueError: Protocol message Schema has no "$ref" field.

### Share and get paid for your tools
You can checkout the following examples to see how to create and share tools on XEntropy.
- [api](https://github.com/ckh112/xentropy/blob/main/docs/api-tool/notebook.ipynb)
- [agent](https://github.com/ckh112/xentropy/blob/main/docs/composite-agents-tool/notebook.ipynb)
- [rag](https://github.com/ckh112/xentropy/blob/main/docs/rag-tool/notebook.ipynb)

## Managing your account

The `Client` class has a few other utilities to access XEntropy platform.

In [8]:
from agentx.client import Client
client = Client(api_key=os.environ.get('XENTROPY_API_KEY'))

# See your account summary and usage data on each tool
client.summary()

In [ ]:
# XEntropy credit can be used to pay for tool usage, if the tool is not free.
# You can top up your XEntropy credit by paying with credit card by navigating to the payment link.
print(client.summary().get('payment_link'))

In [ ]:
# Alternatively you can also top up your XEntropy credit by paying with cryptocurrency.
# We accept USDT, USDC, and DAI on the ethereum blockchain.
# First, register your address.
from eth_account import Account
account = Account.from_key('YOUR_PRIVATE_KEY')
# you can also use mnemonic phrase
# account = Account.from_mnemonic('YOUR_MNEMONIC_PHRASE')
response = client.register_ethereum_address(account=account)
print(response.json())

In [ ]:
# Sometimes you'd like to modify your published tool
# Example 1: Making the tool public to XEntropy platform user
client.modify_tool(
    tool="YOUR_TOOL_NAME",
    key="public",
    value=True,
)

In [ ]:
# Example 2: Tool description acts as a prompt to LLM, and hence it can be optimised through prompt engineering.
# You may want to modify it for improved performance.
# Check out https://github.com/microsoft/LMOps/tree/main/prompt_optimization for inspiration.
client.modify_tool(
    tool="YOUR_TOOL_NAME",
    key="description",
    value="YOU_NEW_DESCRIPTION",
)

In [ ]:
# You can take your tool away anytime.
client.delete_tool(
    tool="YOUR_TOOL_NAME"
)

XEntropy pays 80% of the tool-use revenue to tool developers's `payout` wallet. There is also a 5% bonus paid to the `balance` wallet to incentivize tool developers for consuming other tools on XEntropy.

At this moment we only support withdrawl on the ethereum network for selected stable coins. More withdrawal method is on the way.

In [ ]:
# To withdraw from your payout wallet
client.stable_coin_payout(
    amount=100000,  # amount to withdraw denominated in XEntropy Credit. i.e. 100000 XEntropy Credit = 1 USD
    address="YOUR_ETHEREUM_ADDRESS", # your ethereum network address
    stable_coin='USDT' # choose between ['USDT', 'USDC', 'DAI']
)

In [11]:
dict_to_tranf = {
  'name': 'xentropy--geodesic',
  'description': 'Calculate the earth surface distance between two latitude and longitude coordinate',
  'parameters': {
    'title': 'CoordinatePair',
    'type': 'object',
    'properties': {
      'coordinate_0': {
        '$ref': '#/definitions/Coordinate'
      },
      'coordinate_1': {
        '$ref': '#/definitions/Coordinate'
      }
    },
    'required': [
      'coordinate_0',
      'coordinate_1'
    ],
    'definitions': {
      'Coordinate': {
        'title': 'Coordinate',
        'type': 'object',
        'properties': {
          'latitude': {
            'title': 'Latitude',
            'type': 'number'
          },
          'longitude': {
            'title': 'Longitude',
            'type': 'number'
          }
        },
        'required': [
          'latitude',
          'longitude'
        ]
      }
    }
  }
}

In [12]:
import copy
from typing import Optional, Dict, Any

def replace_key(dictionary, old_key, new_key):
    """
    Recursively replaces all occurrences of the old_key with the new_key in a dictionary.

    Args:
        dictionary (Dict[str, Any]): The input dictionary to modify.
        old_key (str): The key to replace.
        new_key (str): The new key to use.

    Returns:
        Dict[str, Any]: The modified dictionary with replaced keys.
    """
    new_dict = {}
    for key, value in dictionary.items():
        if isinstance(value, dict):
            new_dict[key] = replace_key(value, old_key, new_key)  # Recursively process nested dictionaries
        elif isinstance(value, list):
            new_dict[key] = [replace_key(item, old_key, new_key) if isinstance(item, dict) else item for item in value]
            # Recursively process nested dictionaries within lists
        else:
            new_dict[key] = value
    if old_key in new_dict:
        new_dict[new_key] = new_dict.pop(old_key)  # Replace the old_key with the new_key
    return new_dict

def move_extra_fields_to_properties(dictionary: Dict[str, Any]) -> Dict[str, Any]:
    """
    Moves any field outside the "parameters" key that is in the GAPIC schema inside the "properties" field.
    https://cloud.google.com/vertex-ai/docs/reference/rpc/google.cloud.aiplatform.v1beta1#google.cloud.aiplatform.v1beta1.Schema
    Updates the "required" list with the newly added properties.

    Args:
        dictionary (dict): The input dictionary.

    Returns:
        dict: A modified copy of the input dictionary with extra fields moved to properties.

    Raises:
        None.
    """
    dictionary_copy = copy.deepcopy(dictionary)

    for key in dictionary["parameters"].keys():
        if key not in ["type", "format", "description", "nullable", "items", "enum", "properties", "required", "example"]:
            popped_key = dictionary_copy["parameters"].pop(key)
            dictionary_copy["parameters"]["properties"].update(popped_key)
            del popped_key

    for key in dictionary_copy["parameters"]["properties"].keys():
        if key not in dictionary_copy["parameters"]["required"]:
            dictionary_copy["parameters"]["required"].append(key)

    return dictionary_copy


def pop_parameters(dictionary: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    """
    Pops and returns the value of the "parameters" key from the dictionary, if present.

    Args:
        dictionary (dict): The input dictionary.

    Returns:
        dict or None: The value of the "parameters" key, or None if the key is not present.

    Raises:
        None.
    """
    if "parameters" in dictionary:
        return copy.deepcopy(dictionary).pop("parameters")
    else:
        return None


def pop_properties(dictionary: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    """
    Pops and returns the value of the "properties" key from the dictionary, if present.

    Args:
        dictionary (dict): The input dictionary.

    Returns:
        dict or None: The value of the "properties" key, or None if the key is not present.

    Raises:
        None.
    """
    if "properties" in dictionary:
        return copy.deepcopy(dictionary).pop("properties")
    else:
        return None


def change_field_name_to_description(popped_dict: Dict[str, Any]) -> Dict[str, Any]:
    """
    Changes the name of fields that are not in the specified list to "description" within a nested dictionary.

    Args:
        popped_dict (dict): The input dictionary.

    Returns:
        dict: A modified copy of the input dictionary with field names changed to "description".

    Raises:
        None.
    """
    popped_dict_copy = copy.deepcopy(popped_dict)

    for key in popped_dict.keys():
        for subkey in popped_dict[key].keys():
            if subkey not in ["type", "format", "description", "nullable", "items", "enum", "properties", "required", "example"]:
                popped_dict_copy[key]["description"] = popped_dict_copy[key].pop(subkey)
        popped_dict[key] = popped_dict_copy[key]

    return popped_dict

In [18]:
def transform_openai_tool_to_vertexai_tool(dictionary: dict) -> dict:
    """
    Transforms an OpenAI tool dictionary to a Vertex AI tool dictionary by performing the following steps:
    1. Moves extra fields to properties.
    2. Replaces the key "title" with "description".
    3. Changes field names that arent in the GAPIC Schema to "description" within the properties dictionary.

    From the OpenAI schema, it is assume anything not in the GAPIC schema is a description (e.g. title)

    Args:
        dictionary (dict): The input dictionary.

    Returns:
        dict: The transformed dictionary.
    """
    dictionary_copy = copy.deepcopy(dictionary)

    # Move extra fields to properties and replace "title" with "description"
    dictionary_copy_fields_moved_to_prop = move_extra_fields_to_properties(replace_key(dictionary_copy, "title", "description"))

    # Change field names to "description" within properties dictionary
    dictionary_copy_fields_moved_to_prop["parameters"]["properties"] = change_field_name_to_description(pop_properties(pop_parameters(dictionary_copy_fields_moved_to_prop)))

    return dictionary_copy_fields_moved_to_prop

In [13]:
dict_to_tranf_descr = replace_key(dict_to_tranf, "title", "description")

In [14]:
dict_to_tranf_descr_prop = move_extra_fields_to_properties(replace_key(dict_to_tranf, "title", "description"))

In [16]:
dict_to_tranf_descr_prop["parameters"]["properties"] = change_field_name_to_description(pop_properties(pop_parameters(dict_to_tranf_descr_prop)))

In [17]:
dict_to_tranf_descr_prop

{'name': 'xentropy--geodesic',
 'description': 'Calculate the earth surface distance between two latitude and longitude coordinate',
 'parameters': {'type': 'object',
  'properties': {'coordinate_0': {'description': '#/definitions/Coordinate'},
   'coordinate_1': {'description': '#/definitions/Coordinate'},
   'Coordinate': {'type': 'object',
    'properties': {'latitude': {'type': 'number', 'description': 'Latitude'},
     'longitude': {'type': 'number', 'description': 'Longitude'}},
    'required': ['latitude', 'longitude'],
    'description': 'Coordinate'}},
  'required': ['coordinate_0', 'coordinate_1', 'Coordinate'],
  'description': 'CoordinatePair'}}

In [63]:
def pop_parameters(dictionary):
    if "parameters" in dictionary:
        return dictionary.pop("parameters")
    else:
        return None

def pop_properties(dictionary):
    if "properties" in dictionary:
        return dictionary.pop("properties")
    else:
        return None


def transform_dict(dictionary):
    transformed_dict = {}
    for key, value in dictionary.items():
        if key == 'parameters':
            transformed_dict[key] = transform_parameters(value)
        elif isinstance(value, dict):
            transformed_dict[key] = transform_dict(value)
        else:
            transformed_dict[key] = value
    return transformed_dict


def transform_parameters(parameters):
    transformed_parameters = {}
    transformed_parameters['type'] = parameters['type']
    transformed_parameters['description'] = parameters['title']
    transformed_parameters['properties'] = {}
    transformed_parameters['required'] = parameters['required']
    for prop_key, prop_value in parameters['properties'].items():
        transformed_parameters['properties'][prop_key] = transform_property(prop_value)
    transformed_parameters['properties'].update(transform_dict(parameters['definitions']))
    return transformed_parameters


def transform_property(property_dict):
    transformed_property = {}
    for key, value in property_dict.items():
        if key not in ['type', 'properties', 'required']:
            transformed_property['description'] = value
        else:
            transformed_property[key] = value
    return transformed_property


# Example usage
input_dict = {
    'name': 'xentropy--geodesic',
    'description': 'Calculate the earth surface distance between two latitude and longitude coordinate',
    'parameters': {
        'title': 'CoordinatePair',
        'type': 'object',
        'properties': {
            'coordinate_0': {
                '$ref': '#/definitions/Coordinate'
            },
            'coordinate_1': {
                '$ref': '#/definitions/Coordinate'
            },
            'extra_property': 'This is an extra property',
            'type': 'This should not change',
            'required': 'This should not change',
        },
        'required': [
            'coordinate_0',
            'coordinate_1'
        ],
        'definitions': {
            'Coordinate': {
                'title': 'Coordinate',
                'type': 'object',
                'properties': {
                    'latitude': {
                        'title': 'Latitude',
                        'type': 'number'
                    },
                    'longitude': {
                        'title': 'Longitude',
                        'type': 'number'
                    }
                },
                'required': [
                    'latitude',
                    'longitude'
                ]
            }
        }
    }
}

transformed_dict = transform_dict(input_dict)
print(transformed_dict)

AttributeError: 'str' object has no attribute 'items'